# Road Following - Live demo
在这个笔记本中，我们将使用我们训练的模型，使Jetbot在轨道上平稳地移动。

### Load Trained Model
假设您已经按照“train_model.ipynb”笔记本中的说明将 ``best_steering_model_xy.pth`` 下载到工作站。现在，您应该将模型文件上传到JetBot的笔记本目录中。完成后，该笔记本的目录中应该有一个名为 ``best_steering_model_xy.pth`` 文件。
> 在调用下一个单元格之前，请确保文件已完全上载
执行下面的代码来初始化PyTorch模型。从培训笔记本上看，这应该很熟悉。执行下面的代码来初始化PyTorch模型。从培训笔记本上看，这应该很熟悉。

In [ ]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

接下来，从上传的 ``best_steering_model_xy.pth`` 文件中加载训练过的权重。

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

目前，模型权重位于CPU内存上，执行下面的代码传输到GPU设备。

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### Creating the Pre-Processing Function
### 创建预处理功能
现在已经加载模型，但有一个小问题。我们训练模型的格式与相机的格式不完全匹配。为此，我们需要做一些预处理。这包括以下步骤：
1. 从HWC布局转换为CHW布局
2. 使用与训练期间相同的参数进行规格化（相机提供[0，255]范围值和[0，1]范围的训练加载图像，因此我们需要缩放255.0
3. 将数据从CPU存储器传输到GPU存储器
4. 添加批次维度

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

令人惊叹的！我们现在已经定义了我们的预处理函数，它可以将图像从摄像机格式转换为神经网络输入格式。
现在，让我们开始展示我们的相机。你现在应该已经很熟悉了。

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

我们还将创建我们的机器人实例，我们将需要驱动马达。

In [ ]:
from jetbot import Robot

robot = Robot()

现在，我们将定义滑块来控制JetBot
>注意：我们已经初始化已知配置的滑块值，但是这些值可能不适用于您的数据集，因此请根据您的设置和环境增加或减少滑块

1. 速度控制（speed_gain_slider）：启动JetBot增加 ``speed_gain_slider`` `
2. 转向增益控制（steering_gain_sloder）：如果你看到Jetbot抖动，你需要减小``steering_gain_slider`` 直到它平滑
3. 转向偏压控制（steering_bias_slider）：如果您看到JetBot偏向轨迹的最右侧或最左侧，则应控制此滑块，直到JetBot开始沿着中心的直线或轨迹移动。这就解释了运动偏差和相机偏移

>注意：您应该在上述滑块周围以较低的速度播放，以获得平滑的JetBot道路跟随行为。

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

接下来，让我们展示一些滑块，看看JetBot在想什么。x和y滑块将显示预测的x、y值。

转向滑块将显示我们估计的转向值。请记住，这个值不是目标的实际角度，而是几乎成比例。当实际角度为 ``0`` 时，该值为零，并随实际角度增加/减少。

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

接下来，我们将创建一个函数，每当相机的值发生变化时，该函数将被调用。此函数将执行以下步骤
1. 预处理摄像机图像
2. 执行神经网络
3. 计算近似转向值
4. 使用比例/微分控制（PD）控制电机

In [ ]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

太酷了！我们已经创建神经网络执行函数，现在需要将它附加到相机上进行处理。

我们通过 observe功能来实现这一点。
>警告：此代码将移动机器人！！请确保您的机器人有净空，并且它在乐高或轨道上，您已经收集了数据。道路跟随者应该可以工作，但是神经网络只和它训练的数据一样好！

In [ ]:
camera.observe(execute, names='value')

令人惊叹的！如果你的小车已经接入，现在应该会在每一个新的相机帧上产生新的命令。

你现在可以把JetBot放在乐高或者你收集到数据的轨道上，看看它是否可以跟踪轨道。

如果要停止此行为，可以通过执行下面的代码取消附加此回调。

In [ ]:
camera.unobserve(execute, names='value')
robot.stop()

### 结论
这就是现场演示！希望你在看到你的Jetbot在赛道上沿着公路平稳地移动时玩得开心！!!

如果你的Jetbot跟不上路，试着找出它失败的地方。好处是我们可以为这些故障场景收集更多的数据，JetBot应该会变得更好：）